In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import pathlib
import matplotlib.pyplot as plt
from imutils.contours import sort_contours
import imutils
import cv2
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2

In [2]:
str_data_dir = "extracted_images"
data_dir = pathlib.Path(str_data_dir)

In [3]:
batch_size = 64
img_height = 45
img_width = 45

In [4]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = "training",
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = "validation",
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size)

Found 186139 files belonging to 16 classes.
Using 148912 files for training.
Found 186139 files belonging to 16 classes.
Using 37227 files for validation.


In [5]:
class_names = train_ds.class_names
class_names

['(',
 ')',
 '+',
 '-',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'div',
 'times']

In [6]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(64, 45, 45, 3)
(64,)


In [7]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
print(np.min(first_image), np.max(first_image))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
0.0 1.0


In [8]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [14]:
num_classes = 16
factor = 0.01
rate = 0.3
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(32, 3, activation='relu', padding="same", kernel_regularizer=l2(factor)),
    Dropout(rate),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu', padding="same", kernel_regularizer=l2(factor)),
    Dropout(rate),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu', padding="same", kernel_regularizer=l2(factor)),
    Dropout(rate),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(num_classes, activation='softmax')])

In [15]:
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['accuracy'])

In [16]:
model.fit(
train_ds,
validation_data = val_ds,
epochs = 3)

Epoch 1/3


c:\users\hp\appdata\local\programs\python\python39\lib\site-packages\keras\backend.py:5585: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


2327/2327 [==============================] - 194s 83ms/step - loss: 0.3973 - accuracy: 0.9263 - val_loss: 0.4164 - val_accuracy: 0.9642
Epoch 2/3
2327/2327 [==============================] - 195s 84ms/step - loss: 0.1924 - accuracy: 0.9671 - val_loss: 0.3132 - val_accuracy: 0.9745
Epoch 3/3
2327/2327 [==============================] - 197s 85ms/step - loss: 0.1655 - accuracy: 0.9706 - val_loss: 0.2550 - val_accuracy: 0.9761


In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_2 (Rescaling)     (None, 45, 45, 3)         0         
                                                                 
 conv2d_3 (Conv2D)           (None, 45, 45, 32)        896       
                                                                 
 dropout_3 (Dropout)         (None, 45, 45, 32)        0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 22, 22, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 22, 22, 32)        9248      
                                                                 
 dropout_4 (Dropout)         (None, 22, 22, 32)        0         
                                                      

In [18]:
model.save("mathematical_expressionv7.model")

INFO:tensorflow:Assets written to: mathematical_expressionv7.model\assets


INFO:tensorflow:Assets written to: mathematical_expressionv7.model\assets


In [ ]:
val_loss, val_accuracy = model.evaluate(val_ds)
print(val_loss,val_accuracy)